In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

# Load the dataset
df = pd.read_csv('europedailyprices.csv', parse_dates=['Date'], index_col='Date')

# Calculate returns
returns_df = df.pct_change().dropna()

def check_stationarity(time_series):
    result = adfuller(time_series)
    print("ADF Statistic:", result[0])
    print("p-value:", result[1])
    print("Critical Values:")
    for key, value in result[4].items():
        print(f"\t{key}: {value}")
    return result[1] <= 0.05

# Check stationarity for each index
for col in returns_df.columns:
    print(f"Stationarity test for {col}:")
    is_stationary = check_stationarity(returns_df[col].values)  # Convert to numpy array
    print("Is stationary?", is_stationary)
    print()

# Perform Granger causality tests and select best lag order for each index using AIC and BIC
best_lag_aic = {}
best_lag_bic = {}
best_aic = {}
best_bic = {}

max_lag = 1  # You can adjust the range of lag orders as needed

for col in returns_df.columns:
    best_lag_aic[col] = None
    best_lag_bic[col] = None
    best_aic[col] = float('inf')
    best_bic[col] = float('inf')
    
    for lag in range(1, max_lag + 1):
        model = sm.tsa.stattools.grangercausalitytests(
            returns_df[[col, returns_df.columns[0]]], lag, verbose=False
        )
        p_value = model[lag][0]["ssr_ftest"][1]
        
        # Compute AIC and BIC for the current lag order
        aic = -2 * np.log(p_value) + 2 * lag
        bic = -2 * np.log(p_value) + lag * np.log(len(returns_df))
        
        # Check if current lag order is the best based on AIC and BIC
        if aic < best_aic[col]:
            best_aic[col] = aic
            best_lag_aic[col] = lag
        if bic < best_bic[col]:
            best_bic[col] = bic
            best_lag_bic[col] = lag

print("Best lag order (AIC) for each index:")
print(best_lag_aic)

print("Best lag order (BIC) for each index:")
print(best_lag_bic)


Stationarity test for EUSTX50:
ADF Statistic: -20.838269169111815
p-value: 0.0
Critical Values:
	1%: -3.4324916234641485
	5%: -2.8624862335597316
	10%: -2.5672737039186493
Is stationary? True

Stationarity test for IT40:
ADF Statistic: -28.984187170036318
p-value: 0.0
Critical Values:
	1%: -3.4324888216026417
	5%: -2.862484996057603
	10%: -2.5672730450894954
Is stationary? True

Stationarity test for CAC40:
ADF Statistic: -20.681616158174403
p-value: 0.0
Critical Values:
	1%: -3.4324916234641485
	5%: -2.8624862335597316
	10%: -2.5672737039186493
Is stationary? True

Stationarity test for DAX:
ADF Statistic: -20.371298541102234
p-value: 0.0
Critical Values:
	1%: -3.4324916234641485
	5%: -2.8624862335597316
	10%: -2.5672737039186493
Is stationary? True

Stationarity test for IBEX35:
ADF Statistic: -20.16970987627751
p-value: 0.0
Critical Values:
	1%: -3.4324916234641485
	5%: -2.8624862335597316
	10%: -2.5672737039186493
Is stationary? True

Best lag order (AIC) for each index:
{'EUSTX50'

In [22]:
# Perform Granger causality tests with the best lag orders for each index
granger_results_aic = {}
granger_results_bic = {}

for col in returns_df.columns:
    granger_results_aic[col] = {}
    granger_results_bic[col] = {}
    
    for col2 in returns_df.columns:
        if col != col2:
            model_aic = sm.tsa.stattools.grangercausalitytests(
                returns_df[[col, col2]], best_lag_aic[col], verbose=False
            )
            p_value_aic = model_aic[best_lag_aic[col]][0]["ssr_ftest"][1]
            granger_results_aic[col][col2] = p_value_aic
            
            model_bic = sm.tsa.stattools.grangercausalitytests(
                returns_df[[col, col2]], best_lag_bic[col], verbose=False
            )
            p_value_bic = model_bic[best_lag_bic[col]][0]["ssr_ftest"][1]
            granger_results_bic[col][col2] = p_value_bic

# Print the Granger causality test results using AIC and BIC
print("Granger causality test results (AIC):")
for col in returns_df.columns:
    for col2 in returns_df.columns:
        if col != col2:
            print(f"{col} -> {col2}: p-value = {granger_results_aic[col][col2]}")

print("\nGranger causality test results (BIC):")
for col in returns_df.columns:
    for col2 in returns_df.columns:
        if col != col2:
            print(f"{col} -> {col2}: p-value = {granger_results_bic[col][col2]}")


Granger causality test results (AIC):
EUSTX50 -> IT40: p-value = 0.6710287514408504
EUSTX50 -> CAC40: p-value = 0.2284103589805763
EUSTX50 -> DAX: p-value = 0.2664306069812699
EUSTX50 -> IBEX35: p-value = 1.5659906718030308e-08
IT40 -> EUSTX50: p-value = 0.5312637951812949
IT40 -> CAC40: p-value = 0.3589344479117892
IT40 -> DAX: p-value = 0.29611537504208935
IT40 -> IBEX35: p-value = 0.8913736720232482
CAC40 -> EUSTX50: p-value = 0.044141438435248675
CAC40 -> IT40: p-value = 0.1513562638765392
CAC40 -> DAX: p-value = 0.21701892093611402
CAC40 -> IBEX35: p-value = 0.37715243548018806
DAX -> EUSTX50: p-value = 0.1382301678423924
DAX -> IT40: p-value = 5.054743205007568e-05
DAX -> CAC40: p-value = 0.4328760745261794
DAX -> IBEX35: p-value = 0.7400846226356446
IBEX35 -> EUSTX50: p-value = 0.49799171993294
IBEX35 -> IT40: p-value = 0.4960041965267892
IBEX35 -> CAC40: p-value = 8.616728364575064e-106
IBEX35 -> DAX: p-value = 0.024666528568840624

Granger causality test results (BIC):
EUSTX50

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests

# Load the data from europedailyprices.csv
df = pd.read_csv('europedailyprices.csv', parse_dates=['Date'], index_col='Date')

# Select the columns containing the indices prices
indices_columns = ['EUSTX50', 'CAC40', 'IBEX35', 'DAX', 'IT40']
indices_df = df[indices_columns]

# Calculate the daily returns of the indices
returns_df = indices_df.pct_change().dropna()

# Perform the Granger causality test for each pair of indices
max_lag = 1  # You can adjust this based on your data and requirements

# Replace best_lag_aic and best_lag_bic with the appropriate values
best_lag_aic = {
    'EUSTX50': 1, 'CAC40': 1, 'IBEX35': 1, 'DAX': 1, 'IT40': 1
}

best_lag_bic = {
    'EUSTX50': 1, 'CAC40': 1, 'IBEX35': 1, 'DAX': 1, 'IT40': 1
}

granger_results_aic = {}
granger_results_bic = {}

for col in returns_df.columns:
    granger_results_aic[col] = {}
    granger_results_bic[col] = {}
    
    for col2 in returns_df.columns:
        if col != col2:
            model_aic = sm.tsa.stattools.grangercausalitytests(
                returns_df[[col2, col]], best_lag_aic[col], verbose=False
            )
            f_stat_aic = model_aic[best_lag_aic[col]][0]["ssr_ftest"][0]
            p_value_aic = model_aic[best_lag_aic[col]][0]["ssr_ftest"][1]
            granger_results_aic[col][col2] = (f_stat_aic, p_value_aic)
            
            model_bic = sm.tsa.stattools.grangercausalitytests(
                returns_df[[col2, col]], best_lag_bic[col], verbose=False
            )
            f_stat_bic = model_bic[best_lag_bic[col]][0]["ssr_ftest"][0]
            p_value_bic = model_bic[best_lag_bic[col]][0]["ssr_ftest"][1]
            granger_results_bic[col][col2] = (f_stat_bic, p_value_bic)

# Print the Granger causality test results using AIC and BIC
print("Granger causality test results (AIC):")
for col in returns_df.columns:
    for col2 in returns_df.columns:
        if col != col2:
            f_stat, p_value = granger_results_aic[col][col2]
            print(f"{col} -> {col2}: F-statistic = {f_stat:.4f}, p-value = {p_value:.4f}")

print("\nGranger causality test results (BIC):")
for col in returns_df.columns:
    for col2 in returns_df.columns:
        if col != col2:
            f_stat, p_value = granger_results_bic[col][col2]
            print(f"{col} -> {col2}: F-statistic = {f_stat:.4f}, p-value = {p_value:.4f}")


Granger causality test results (AIC):
EUSTX50 -> CAC40: F-statistic = 4.0545, p-value = 0.0441
EUSTX50 -> IBEX35: F-statistic = 0.4593, p-value = 0.4980
EUSTX50 -> DAX: F-statistic = 2.1987, p-value = 0.1382
EUSTX50 -> IT40: F-statistic = 0.3921, p-value = 0.5313
CAC40 -> EUSTX50: F-statistic = 1.4513, p-value = 0.2284
CAC40 -> IBEX35: F-statistic = 516.5309, p-value = 0.0000
CAC40 -> DAX: F-statistic = 0.6153, p-value = 0.4329
CAC40 -> IT40: F-statistic = 0.8419, p-value = 0.3589
IBEX35 -> EUSTX50: F-statistic = 32.1425, p-value = 0.0000
IBEX35 -> CAC40: F-statistic = 0.7802, p-value = 0.3772
IBEX35 -> DAX: F-statistic = 0.1101, p-value = 0.7401
IBEX35 -> IT40: F-statistic = 0.0187, p-value = 0.8914
DAX -> EUSTX50: F-statistic = 1.2355, p-value = 0.2664
DAX -> CAC40: F-statistic = 1.5246, p-value = 0.2170
DAX -> IBEX35: F-statistic = 5.0521, p-value = 0.0247
DAX -> IT40: F-statistic = 1.0920, p-value = 0.2961
IT40 -> EUSTX50: F-statistic = 0.1804, p-value = 0.6710
IT40 -> CAC40: F-sta

In [2]:
# Perform Granger causality tests with the best lag orders for each index
granger_results_aic = {}
granger_results_bic = {}

for col in returns_df.columns:
    granger_results_aic[col] = {}
    granger_results_bic[col] = {}
    
    for col2 in returns_df.columns:
        if col != col2:
            model_aic = sm.tsa.stattools.grangercausalitytests(
                returns_df[[col2, col]], max_lag, verbose=False
            )
            best_lag_aic = min(model_aic.keys(), key=lambda k: model_aic[k][0]['ssr_ftest'][1])
            f_stat_aic = model_aic[best_lag_aic][0]["ssr_ftest"][0]
            p_value_aic = model_aic[best_lag_aic][0]["ssr_ftest"][1]
            granger_results_aic[col][col2] = (f_stat_aic, p_value_aic)
            
            model_bic = sm.tsa.stattools.grangercausalitytests(
                returns_df[[col2, col]], max_lag, verbose=False
            )
            best_lag_bic = min(model_bic.keys(), key=lambda k: model_bic[k][0]['ssr_ftest'][1])
            f_stat_bic = model_bic[best_lag_bic][0]["ssr_ftest"][0]
            p_value_bic = model_bic[best_lag_bic][0]["ssr_ftest"][1]
            granger_results_bic[col][col2] = (f_stat_bic, p_value_bic)

# Print the Granger causality test results using AIC and BIC
print("Granger causality test results (AIC):")
for col in returns_df.columns:
    for col2 in returns_df.columns:
        if col != col2:
            f_stat, p_value = granger_results_aic[col][col2]
            print(f"{col} -> {col2}: F-statistic = {f_stat:.4f}, p-value = {p_value:.4f}")

print("\nGranger causality test results (BIC):")
for col in returns_df.columns:
    for col2 in returns_df.columns:
        if col != col2:
            f_stat, p_value = granger_results_bic[col][col2]
            print(f"{col} -> {col2}: F-statistic = {f_stat:.4f}, p-value = {p_value:.4f}")


Granger causality test results (AIC):
EUSTX50 -> CAC40: F-statistic = 4.0545, p-value = 0.0441
EUSTX50 -> IBEX35: F-statistic = 0.4593, p-value = 0.4980
EUSTX50 -> DAX: F-statistic = 2.1987, p-value = 0.1382
EUSTX50 -> IT40: F-statistic = 0.3921, p-value = 0.5313
CAC40 -> EUSTX50: F-statistic = 1.4513, p-value = 0.2284
CAC40 -> IBEX35: F-statistic = 516.5309, p-value = 0.0000
CAC40 -> DAX: F-statistic = 0.6153, p-value = 0.4329
CAC40 -> IT40: F-statistic = 0.8419, p-value = 0.3589
IBEX35 -> EUSTX50: F-statistic = 32.1425, p-value = 0.0000
IBEX35 -> CAC40: F-statistic = 0.7802, p-value = 0.3772
IBEX35 -> DAX: F-statistic = 0.1101, p-value = 0.7401
IBEX35 -> IT40: F-statistic = 0.0187, p-value = 0.8914
DAX -> EUSTX50: F-statistic = 1.2355, p-value = 0.2664
DAX -> CAC40: F-statistic = 1.5246, p-value = 0.2170
DAX -> IBEX35: F-statistic = 5.0521, p-value = 0.0247
DAX -> IT40: F-statistic = 1.0920, p-value = 0.2961
IT40 -> EUSTX50: F-statistic = 0.1804, p-value = 0.6710
IT40 -> CAC40: F-sta